This notebook prepare the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
017bb1e (Mon Jun 18 14:58:59 2018 +0100)
This notebook was executed on: 
2019-04-02 17:49:46.997545


In [2]:
import numpy as np
import os
os.environ['LOG_LEVEL'] = 'INFO'

from astropy.table import Table, Column

from herschelhelp_internal.utils import gen_help_id

from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

In [3]:
SUFFIX="20190402"
master_catalogue = Table.read("../../dmu32/dmu32_COSMOS/data/COSMOS_{}_cigale.fits".format(SUFFIX))


## id renaming

Cigale requires the ids to be in the current format unlike the old ids used because COSMOS was processed first so we add new ones and create a cross ident for the final join to return them to the orginal values.

In [4]:
master_catalogue['help_id', 'old_help_id'].write(
    "./data/master_list_help_cigale_cross_ident_cosmos_{}.fits".format(SUFFIX), overwrite=True)


In [5]:
#For some reason there is an issue with the flag dtypes
for col in master_catalogue.colnames:
    if col.startswith('flag') and master_catalogue[col].dtype == 'float64':
        master_catalogue[col] = master_catalogue[col].astype('bool')
    

In [6]:
#Remove infinities and negative values
for col in master_catalogue.colnames:
    if col.startswith('f_') or col.startswith('ferr_') or col.startswith('m_') or col.startswith('merr_'):
        mask = np.isinf(master_catalogue[col]) | (master_catalogue[col] < 0)
        #print("Problems in {}: {}".format(col,np.sum(mask)))
        master_catalogue[col][mask] = np.nan
        mask = np.isinf(master_catalogue[col]) | (master_catalogue[col] < 0)
        #print("probs after in {}: {}".format(col,np.sum(mask))  )    

/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)


In [7]:
len(master_catalogue)

2599374

# Best sources

Define a good far-IR measurement as:
- an existing flux in the band;
- the flag from XID+ must not be set;
- the signal to noise ratio must be over 2.

In [8]:
good = {}
for band in ['pacs_green', 'pacs_red', 'spire_250', 'spire_350', 'spire_500']:
    good[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]) )
    good[band][good[band]] &= (master_catalogue[good[band]]['f_{}'.format(band)] /
                              master_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)

We will keep only sources with at leat 2 good far-IR measurements (we may actually use less sources are not all may have a redshift).

In [9]:
combined_good = np.sum(list(good.values()), axis=0) >= 2
all_xid =  np.sum(list(good.values()), axis=0) >= 0

In [10]:
print("Number of good sources: {}".format(np.sum(combined_good)))

Number of good sources: 29129


In [11]:
# Only sources with at least two optical and at least two near infrared detections
optnir = ((master_catalogue['flag_optnir_det'] == 3) 
          | (master_catalogue['flag_optnir_det'] == 7))

# Main catalogue for CIGALE

In [12]:
best_catalogue = master_catalogue[combined_good].copy()

In [13]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [14]:
# Convertion to CIGALE format
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 2436 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2253 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 488 sources, the band suprime_ib427 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1294 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 274 sources, the band suprime_ib464 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.

## Band selection

We want to use only one filter for similar bands. We define an order of preference and set to NaN the flux in the lower prefered bands when a prefered band is available. Some band may have a 0 flux, we set there values to NaN.

In [15]:
u_bands = ["megacam_u",              "omegacam_u"]
g_bands = ["megacam_g", "suprime_g", "omegacam_g", "decam_g", "gpc1_g"]
r_bands = ["megacam_r", "suprime_r", "omegacam_r", "decam_r", "gpc1_r"]
i_bands = ["megacam_i", "suprime_i", "omegacam_i",            "gpc1_i"]
z_bands = ["megacam_z", "suprime_z",               "decam_z", "gpc1_z"]
y_bands = [             "suprime_y",                          "gpc1_y", "vista_y", "ukidss_y"]
j_bands = ["vista_j", "wircam_j",   "ukidss_j"  ]
h_bands = ["vista_h",  "wircam_h",  "ukidss_h"  ]
ks_bands = ["vista_ks", "wircam_ks" ]

def remove_unneeded_fluxes(list_of_bands):
    for band_idx, band in enumerate(list_of_bands[:-1]):
        mask = ~np.isnan(best_catalogue[band])
        for lower_band in list_of_bands[band_idx+1:]:
            best_catalogue[lower_band][mask] = np.nan
            best_catalogue["{}_err".format(lower_band)][mask] = np.nan

In [16]:
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)
remove_unneeded_fluxes(j_bands)
remove_unneeded_fluxes(h_bands)
remove_unneeded_fluxes(ks_bands)

In [17]:
best_catalogue.write("data_tmp/COSMOS_cigale_best_extcor_{}.fits".format(SUFFIX), overwrite=True)

## All XID+ objects

In [18]:
best_catalogue = master_catalogue[all_xid].copy()

In [19]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [20]:
# Convertion to CIGALE format
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 123896 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 114820 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 42625 sources, the band suprime_ib427 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 81401 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 24427 sources, the band suprime_ib464 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:hers

In [21]:
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)
remove_unneeded_fluxes(j_bands)
remove_unneeded_fluxes(h_bands)
remove_unneeded_fluxes(ks_bands)

In [22]:
best_catalogue.write("data_tmp/COSMOS_cigale_allxid_extcor_{}.fits".format(SUFFIX), overwrite=True)

# Make catalogue of all photo-z objects

In [23]:
best_catalogue = master_catalogue[optnir].copy()

In [24]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [25]:
# Convertion to CIGALE format
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 115771 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 107289 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 38631 sources, the band suprime_ib427 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 76043 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 21169 sources, the band suprime_ib464 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:hers

In [26]:
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)
remove_unneeded_fluxes(j_bands)
remove_unneeded_fluxes(h_bands)
remove_unneeded_fluxes(ks_bands)

In [27]:
best_catalogue.write("data_tmp/COSMOS_cigale_optnir_extcor_{}.fits".format(SUFFIX), overwrite=True)

# Catalogue using spectroscopic redshift

In [28]:
best_catalogue = master_catalogue[optnir].copy()

In [29]:
best_catalogue.remove_column("redshift")
best_catalogue["zspec"].name = "redshift"

In [30]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

In [31]:
print("Number of sources with z-spec: {}".format(len(best_catalogue)))

Number of sources with z-spec: 39892


In [32]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [33]:
# Convertion to CIGALE format
os.environ['LOG_LEVEL'] = 'INFO'
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 94 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 88 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 22 sources, the band suprime_ib427 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 55 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 13 sources, the band suprime_ib464 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external

In [34]:
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)
remove_unneeded_fluxes(j_bands)
remove_unneeded_fluxes(h_bands)
remove_unneeded_fluxes(ks_bands)

In [35]:
best_catalogue.write("data_tmp/COSMOS_cigale_optnir_extcor_zspec_{}.fits".format(SUFFIX), overwrite=True)